In [32]:
import pandas as pd
import os
import pickle

from source.transformer_forecasting import TransformerForecasting

#### 1. Config parameters

In [33]:
# Dataset specific parameters
dataset_name = 'awsce'                              # Dataset name
input_col = 'price'                                 # Dataset column of the independent variable
output_col = 'demand'                               # Dataset column of the dependent variable
result_col = 'revenue'                              # Result column of the objective function
item_id = "item_277"                                # Item ID used in scenario generation
store_id = "store_001"                              # Store ID used in scenario generation
test_size = 0.2                                     # Test size for train-test split

# Transformer parameters
num_layers = 4                                      # range: 2-6
size_layer = 128                                    # range: 64-256
embedded_size = 128                                 # range: 64-256
epochs = 500                                        # range: 200-500 w/ early stopping
dropout_rate = 0.2                                  # range: 0.1-0.3
learning_rate = 0.0001                              # range: 0.0001-0.001
batch_size = 64                                     # 

#### 2. Load data

In [34]:
# Load dataset

# AWS Consumer Electronics Dataset
df = pd.read_csv("datasets/aws_ce_dataset.csv", index_col="timestamp", parse_dates=True)

# Filter dataset by item id and store id
data = df[(df["item_id"] == item_id) & (df["store_id"] == store_id)][:-forecast_length]
data_test = df[(df["item_id"] == item_id) & (df["store_id"] == store_id)][-forecast_length-1:]

In [35]:
# Dataset train-test split

unique_item_ids = df['item_id'].unique()
test_size = int(0.2*len(unique_item_ids))

# complete dataset
train_df = df[df['item_id'].isin(unique_item_ids[:-test_size])]
test_df = df[df['item_id'].isin(unique_item_ids[-test_size:])]

#### 3. Train model

In [36]:
model_input = TransformerForecasting(size_layer=size_layer, embedded_size=size_layer, output_size=1, dropout_rate=dropout_rate)
model_output = TransformerForecasting(size_layer=size_layer, embedded_size=size_layer, output_size=1, dropout_rate=dropout_rate)

Initializing Transformer Model
Initializing Transformer Model


In [37]:
def train_model(df, model, col, dataset_name="dataset"):
    """Training the transformer model on the training set"""

    train_prices = df[col].dropna().values
    model_file_path = f"trained-models/transformer-{dataset_name}-{col}.pkl"

    if os.path.exists(model_file_path):
        print(f"Trained model already exists -> loading model {model_file_path}")
        with open(model_file_path, 'rb') as file:
            model = pickle.load(file)
    else:
        print(f"Trained model not found -> training a new model {model_file_path}")
        # Train the model
        model.train(train_prices, batch_size, epochs, learning_rate)
        
        # Save the trained model
        os.makedirs(os.path.dirname(model_file_path), exist_ok=True)
        with open(model_file_path, 'wb') as file:
            pickle.dump(model, file)
        print("Model trained and saved successfully.")

    return model

In [38]:
model_input = train_model(train_df, model_input, input_col, dataset_name)
model_output = train_model(train_df, model_output, output_col, dataset_name)

Trained model already exists -> loading model trained-models/transformer-awsce-price.pkl
Trained model already exists -> loading model trained-models/transformer-awsce-demand.pkl
